# Cache Analysis Notebook

# Setup Environment

In [ ]:
from timeit import default_timer as timer
import numpy as np
import holoviews as hv
import datashader as ds
import pandas as pd
import holoviews.operation.datashader as hd
from holoviews.operation.datashader import aggregate, shade, datashade, dynspread, stack
from datashader import transfer_functions as tf
from holoviews.operation import decimate
from IPython.core.display import display, HTML
from datashader.colors import Sets1to3 # default datashade() and shade() color cycle
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
display(HTML("<style>.container { width:100% !important; height:100% important}</style>"))
hv.extension('bokeh')
hv.notebook_extension('bokeh')
decimate.max_samples=1000
dynspread.max_px=20
dynspread.threshold=0.5
plot_width  = int(750)
plot_height = int(plot_width//1.2)




# Read Data

In [ ]:
import re
import sys
offsets = {}
data = {}
memory_ops = {}
n_lines = 0
n_max_lines = sys.maxsize
file_list = ["MEMORY_DUMPS/memory.txt"]
regex_1 = r"TID[0x]*([abcdef0-9]+).*INS(0x[abcdef0-9]+).*(R|W)0x([abcdef0-9]+)"
run_FIFO = False
run_LRU = True
start = timer()
for file in file_list:
    file_name = re.split('\.',file)[0]
    file_name = re.sub("/","-",file_name)
    with open(file,'r') as file:
        for line in file:
            n_lines += 1
            if n_lines > n_max_lines:
                break
            match = re.match(regex_1,line)
            if match:
                tid = int(match.group(1), 16)
                ins = match.group(2)
                memory_op = match.group(3)
                data_name = file_name
                thread_name = str(tid)
                address = int(match.group(4), 16)
                if data_name not in data:
                    data[data_name] = {}
                    offsets[data_name] = {}
                    memory_ops[data_name] = {}
                if thread_name not in data[data_name]:
                    data[data_name][thread_name] = []
                    memory_ops[data_name][thread_name] = []
                    offsets[data_name][thread_name] = address
                    print("data name: ", data_name, ", thread name: ", thread_name, ", memory op: ", memory_op, ", address: ", address)
                memory_ops[data_name][thread_name].append(memory_op)
                data[data_name][thread_name].append(address)
end = timer()
print('File read complete: ' + str(end - start))
min_offset = {}
for data_name in data:
    if data_name not in min_offset:
        min_offset[data_name] = sys.maxsize
        for thread_name in data[data_name]:
            offset = offsets[data_name][thread_name]
            if offset < min_offset[data_name]:
                min_offset[data_name] = offset
max_len = {}
n_data = {}
thread_names = {}
for data_name in data:
    offset = min_offset[data_name]
    n_data[data_name] = {}
    max_len[data_name] = 0
    thread_names[data_name] = []
    for thread_name in data[data_name]:
        thread_names[data_name].append(thread_name)
        n_data[data_name][thread_name] = len(data[data_name][thread_name])
        max_len[data_name] = max(max_len[data_name], n_data[data_name][thread_name])
        for i in range(0, len(data[data_name][thread_name])):
            address = data[data_name][thread_name][i]
            data[data_name][thread_name][i] -= offset
                
for data_name in data:
    new_data = []
    mem_ops = []
    for i in range(max_len[data_name]):
        new_data.append({})
        mem_ops.append({})
        for thread_name in data[data_name]:
            if i < n_data[data_name][thread_name]:
                new_data[i][thread_name] = data[data_name][thread_name][i]
                mem_ops[i][thread_name] = memory_ops[data_name][thread_name][i]
    data[data_name] = new_data
    memory_ops[data_name] = mem_ops
end = timer()
print("Data input complete: " + str(n_lines) + " lines. " + str(end - start) + " s")

# LRU Cache Simulation

In [ ]:
%%opts RGB [width=plot_width, height=plot_height] {+axiswise}
if run_LRU:
    from collections import OrderedDict, defaultdict

    start = timer()

    cache_simulation_type = "LRU"

    class CacheLine:

        def __init__(self, id, previous_cache_line, next_cache_line, MEMORY_OP="", TID=-1, modified=False):
            self.id = id
            self.previous_cache_line = previous_cache_line
            self.next_cache_line = next_cache_line
            self.last_TID = TID
            self.last_MEMORY_OP = MEMORY_OP
            self.MODIFIED = modified

    cache_scale_factor = 1

    L1_cache_size = 2**15 // cache_scale_factor
    L1_cache_lines = OrderedDict()
    L1_cache_misses = OrderedDict()
    L2_cache_size = 2**18 // cache_scale_factor
    L2_cache_lines = OrderedDict()
    L2_cache_misses = OrderedDict()
    L3_cache_size = 10 * (2**21) // cache_scale_factor
    L3_cache_lines = OrderedDict()
    L3_cache_misses = OrderedDict()
    cache_line_size = 64
    cache_misses = OrderedDict()
    first_L1_cache_line = OrderedDict()
    last_L1_cache_line = OrderedDict()
    first_L2_cache_line = OrderedDict()
    last_L2_cache_line = OrderedDict()
    first_L3_cache_line = OrderedDict()
    last_L3_cache_line = OrderedDict()
    for data_name in data:
        if data_name not in cache_misses:
            cache_misses[data_name] = OrderedDict()
        if data_name not in L1_cache_lines:
            L1_cache_lines[data_name] = OrderedDict()
            first_L1_cache_line[data_name] = OrderedDict()
            last_L1_cache_line[data_name] = OrderedDict()
        if data_name not in L2_cache_lines:
            L2_cache_lines[data_name] = OrderedDict()
            first_L2_cache_line[data_name] = OrderedDict()
            last_L2_cache_line[data_name] = OrderedDict()
        if data_name not in L3_cache_lines:
            L3_cache_lines[data_name] = OrderedDict()
            first_L3_cache_line[data_name] = None
            last_L3_cache_line[data_name] = None
        for thread_name in sorted(thread_names[data_name]):
            if thread_name not in cache_misses[data_name]:
                cache_misses[data_name][thread_name] = []
            if thread_name not in L1_cache_lines[data_name]:
                L1_cache_lines[data_name][thread_name] = OrderedDict()
                first_L1_cache_line[data_name][thread_name] = None
                last_L1_cache_line[data_name][thread_name] = None
            if thread_name not in L2_cache_lines[data_name]:
                L2_cache_lines[data_name][thread_name] = OrderedDict()
                first_L2_cache_line[data_name][thread_name] = None
                last_L2_cache_line[data_name][thread_name] = None
        for i in range(max_len[data_name]):
            for thread_name in data[data_name][i]:
                address = data[data_name][i][thread_name]
                memory_op = memory_ops[data_name][i][thread_name]
                cache_line = address // cache_line_size
                # Record Read/Write access
                if memory_op == "W":
                    for other_thread_name in data[data_name][i]:
                        if thread_name == other_thread_name:
                            break
                        if cache_line in L1_cache_lines[data_name][other_thread_name]:
                            L1_cache_lines[data_name][other_thread_name][cache_line].MODIFIED = True
                        if cache_line in L2_cache_lines[data_name][other_thread_name]:
                            L2_cache_lines[data_name][other_thread_name][cache_line].MODIFIED = True
            for thread_name in data[data_name][i]:
                address = data[data_name][i][thread_name]
                memory_op = memory_ops[data_name][i][thread_name]
                cache_line = address // cache_line_size
                cache_misses[data_name][thread_name].append(1)    
                modified = False
                shared = False
                # L1 Cache Miss
                if cache_line not in L1_cache_lines[data_name][thread_name]:
                    # L1 Cache full - remove least recently used line
                    if cache_line_size * len(L1_cache_lines[data_name][thread_name]) == L1_cache_size:
                        dead_cache_line = last_L1_cache_line[data_name][thread_name]
                        last_L1_cache_line[data_name][thread_name] = dead_cache_line.next_cache_line
                        last_L1_cache_line[data_name][thread_name].previous_cache_line = None
                        del L1_cache_lines[data_name][thread_name][dead_cache_line.id]
                        L1_cache_lines[data_name][thread_name][last_L1_cache_line[data_name][thread_name].id] = last_L1_cache_line[data_name][thread_name]
                    next_L1_cache_line = None
                    # Create new L1 Cache Line
                    new_cache_line = CacheLine(cache_line, first_L1_cache_line[data_name][thread_name], next_L1_cache_line, thread_name)
                    L1_cache_lines[data_name][thread_name][cache_line] = new_cache_line
                    previous_cache_line = first_L1_cache_line[data_name][thread_name]
                    if previous_cache_line:
                        previous_cache_line.next_cache_line = new_cache_line
                        first_L1_cache_line[data_name][thread_name] = new_cache_line
                    else:
                        first_L1_cache_line[data_name][thread_name] = new_cache_line
                        last_L1_cache_line[data_name][thread_name] = new_cache_line
                    cache_misses[data_name][thread_name][-1] = 2
                # L1 Cache Hit
                else:
                    # Extract this cache line from the list, patch up the hole in the list, and add the cache line to the end of the list
                    this_cache_line = L1_cache_lines[data_name][thread_name][cache_line]
                    first_cache_line = first_L1_cache_line[data_name][thread_name]
                    if this_cache_line != first_cache_line:
                        if this_cache_line.next_cache_line:
                            if this_cache_line.previous_cache_line:
                                this_cache_line.previous_cache_line.next_cache_line = this_cache_line.next_cache_line
                                this_cache_line.next_cache_line.previous_cache_line = this_cache_line.previous_cache_line
                            else:
                                last_cache_line = this_cache_line.next_cache_line
                                last_cache_line.previous_cache_line = None
                                last_L1_cache_line[data_name][thread_name] = last_cache_line
                                first_cache_line.next_cache_line = this_cache_line
                                this_cache_line.previous_cache_line = first_cache_line
                                first_L1_cache_line[data_name][thread_name] = this_cache_line
                # L2 Cache Miss
                if cache_line not in L2_cache_lines[data_name][thread_name]:
                    if cache_line_size * len(L2_cache_lines[data_name][thread_name]) == L2_cache_size:
                        dead_cache_line = last_L2_cache_line[data_name][thread_name]
                        last_L2_cache_line[data_name][thread_name] = dead_cache_line.next_cache_line
                        last_L2_cache_line[data_name][thread_name].previous_cache_line = None
                        del L2_cache_lines[data_name][thread_name][dead_cache_line.id]
                        L2_cache_lines[data_name][thread_name][last_L2_cache_line[data_name][thread_name].id] = last_L2_cache_line[data_name][thread_name]
                    next_L2_cache_line = None
                    new_cache_line = CacheLine(cache_line, first_L2_cache_line[data_name][thread_name], next_L2_cache_line, thread_name)
                    L2_cache_lines[data_name][thread_name][cache_line] = new_cache_line
                    previous_cache_line = first_L2_cache_line[data_name][thread_name]
                    if previous_cache_line:
                        previous_cache_line.next_cache_line = new_cache_line
                        first_L2_cache_line[data_name][thread_name] = new_cache_line
                    else:
                        first_L2_cache_line[data_name][thread_name] = new_cache_line
                        last_L2_cache_line[data_name][thread_name] = new_cache_line
                    cache_misses[data_name][thread_name][-1] = 3
                # L2 Cache Hit
                else:
                    this_cache_line = L2_cache_lines[data_name][thread_name][cache_line]
                    first_cache_line = first_L2_cache_line[data_name][thread_name]
                    if this_cache_line != first_cache_line:
                        if this_cache_line.next_cache_line:
                            if this_cache_line.previous_cache_line:
                                this_cache_line.previous_cache_line.next_cache_line = this_cache_line.next_cache_line
                                this_cache_line.next_cache_line.previous_cache_line = this_cache_line.previous_cache_line
                            else:
                                last_cache_line = this_cache_line.next_cache_line
                                last_cache_line.previous_cache_line = None
                                last_L2_cache_line[data_name][thread_name] = last_cache_line
                                first_cache_line.next_cache_line = this_cache_line
                                this_cache_line.previous_cache_line = first_cache_line
                                first_L2_cache_line[data_name][thread_name] = this_cache_line
                # L3 Cache Miss
                if cache_line not in L3_cache_lines[data_name]:
                    if cache_line_size * len(L3_cache_lines[data_name]) == L3_cache_size:
                        dead_cache_line = last_L3_cache_line[data_name]
                        last_L3_cache_line[data_name] = dead_cache_line.next_cache_line
                        last_L3_cache_line[data_name].previous_cache_line = None
                        del L3_cache_lines[data_name][dead_cache_line.id]
                        L3_cache_lines[data_name][last_L3_cache_line[data_name].id] = last_L3_cache_line[data_name]
                    next_L3_cache_line = None
                    new_cache_line = CacheLine(cache_line, first_L3_cache_line[data_name], next_L3_cache_line, thread_name)
                    L3_cache_lines[data_name][cache_line] = new_cache_line
                    previous_cache_line = first_L3_cache_line[data_name]
                    if previous_cache_line:
                        previous_cache_line.next_cache_line = new_cache_line
                        first_L3_cache_line[data_name] = new_cache_line
                    else:
                        first_L3_cache_line[data_name] = new_cache_line
                        last_L3_cache_line[data_name] = new_cache_line
                    cache_misses[data_name][thread_name][-1] = 4
                # L3 Cache Hit
                else:
                    if L3_cache_lines[data_name][cache_line].last_TID != thread_name:
                        if cache_misses[data_name][thread_name][-1] == 3:
                            shared = True
                    L3_cache_lines[data_name][cache_line].last_TID = thread_name
                    this_cache_line = L3_cache_lines[data_name][cache_line]
                    first_cache_line = first_L3_cache_line[data_name]
                    if this_cache_line != first_cache_line:
                        if this_cache_line.next_cache_line:
                            if this_cache_line.previous_cache_line:
                                this_cache_line.previous_cache_line.next_cache_line = this_cache_line.next_cache_line
                                this_cache_line.next_cache_line.previous_cache_line = this_cache_line.previous_cache_line
                            else:
                                last_cache_line = this_cache_line.next_cache_line
                                last_cache_line.previous_cache_line = None
                                last_L3_cache_line[data_name] = last_cache_line
                                first_cache_line.next_cache_line = this_cache_line
                                this_cache_line.previous_cache_line = first_cache_line
                                first_L3_cache_line[data_name] = this_cache_line
                # L1 cache line was modified by another thread
                if L1_cache_lines[data_name][thread_name][cache_line].MODIFIED:
                    modified = True
                    L1_cache_lines[data_name][thread_name][cache_line].MODIFIED = False
                # L2 cache line was modified by another thread
                if L2_cache_lines[data_name][thread_name][cache_line].MODIFIED:
                    modified = True
                    L2_cache_lines[data_name][thread_name][cache_line].MODIFIED = False
                # Mark cache line as modified
                if modified:
                    cache_misses[data_name][thread_name][-1] = 5
                # L3 Cache line was previously accessed by another thread. Mark as shared cache line
                if shared:
                    cache_misses[data_name][thread_name][-1] += 10
                # Mark as writ operation
                if memory_op == "W":
                    cache_misses[data_name][thread_name][-1] += 100
                        
    summary_data = OrderedDict()
    for data_name in cache_misses:
        summary_data[data_name] = OrderedDict()
        summary_data[data_name]["All"] = defaultdict(int)
        for thread_name in cache_misses[data_name]:
            summary_data[data_name][thread_name] = defaultdict(int)
            summary_data[data_name][thread_name]["Cache_References"] = len(cache_misses[data_name][thread_name])
            for i in range(len(cache_misses[data_name][thread_name])):
                cache_level = cache_misses[data_name][thread_name][i]
                l = cache_level
                if l // 100 == 0:
                    summary_data[data_name][thread_name]["Reads"] += 1
                else: 
                    summary_data[data_name][thread_name]["Writes"] += 1
                l = (l % 100)
                if l // 10 == 1:
                    summary_data[data_name][thread_name]["L3_Hits_Shared"] += 1
                l = (l % 10)
                if l == 1:
                    summary_data[data_name][thread_name]["L1_Cache_Hits"] += 1
                elif l == 2:
                    summary_data[data_name][thread_name]["L2_Cache_Hits"] += 1
                elif l == 3:
                    summary_data[data_name][thread_name]["L3_Cache_Hits"] += 1
                elif l == 4:
                    summary_data[data_name][thread_name]["L3_Cache_Misses"] += 1
                elif l == 5:
                    summary_data[data_name][thread_name]["Cache_Hits_Modified"] += 1
        for thread_name in summary_data[data_name]:
            summary_data[data_name]["All"]["Cache_References"] += summary_data[data_name][thread_name]["Cache_References"]
            summary_data[data_name]["All"]["Reads"] += summary_data[data_name][thread_name]["Reads"]
            summary_data[data_name]["All"]["Writes"] += summary_data[data_name][thread_name]["Writes"]
            summary_data[data_name]["All"]["L1_Cache_Hits"] += summary_data[data_name][thread_name]["L1_Cache_Hits"]
            summary_data[data_name]["All"]["L2_Cache_Hits"] += summary_data[data_name][thread_name]["L2_Cache_Hits"]
            summary_data[data_name]["All"]["L3_Cache_Hits"] += summary_data[data_name][thread_name]["L3_Cache_Hits"]
            summary_data[data_name]["All"]["L3_Cache_Misses"] += summary_data[data_name][thread_name]["L3_Cache_Misses"]
            summary_data[data_name]["All"]["Cache_Hits_Modified"] += summary_data[data_name][thread_name]["Cache_Hits_Modified"]
            summary_data[data_name]["All"]["L3_Hits_Shared"] += summary_data[data_name][thread_name]["L3_Hits_Shared"]
    for data_name in summary_data:
        for thread_name in sorted(summary_data[data_name].keys()):
            for label in sorted(summary_data[data_name][thread_name].keys()):
                val = summary_data[data_name][thread_name][label]
                total = summary_data[data_name][thread_name]["Cache_References"]
                pc = '{0:.2F}'.format(100.0 * float(val) / float(total))
                print(data_name + "---" + thread_name + "---" + label + ": " + str(val) + " (" + pc + "%)")

    end = timer()

    print('LRU cache simulation complete: ' + str(end - start) + " s")
else:
    print('LRU cache simulation skipped')



# Plot Cache Sim Totals: All Threads

In [ ]:
%%opts Bars [width=plot_width, height=plot_height] {+axiswise}
print("Cache simulation: " + cache_simulation_type)
start = timer()
d_s = {'data_name':[], 'thread_id': [], 'property': [], 'value': []}
for data_name in sorted(summary_data.keys()):
    for thread_name in sorted(summary_data[data_name].keys()):
        for property_name in sorted(summary_data[data_name][thread_name].keys()):
            d_s['data_name'].append(data_name)
            d_s['thread_id'].append(thread_name)
            d_s['property'].append(property_name)
            d_s['value'].append(summary_data[data_name][thread_name][property_name])
df_s = pd.DataFrame(data=d_s)
plots = {}
for data_name in sorted(summary_data.keys()):
    for property_name in sorted(summary_data[data_name]["All"].keys()):
        plots[(data_name, property_name)] = hv.Bars(df_s.loc[(df_s['data_name'] == data_name) & (df_s['property'] == property_name)], kdims=['thread_id'], vdims=['value'])
holomap = hv.HoloMap(plots, kdims=['data_name','property_name'])
end = timer()
print("Time: " + str(end - start) + " s")
holomap

# Plot Cache Sim Totals: All Data

In [ ]:
%%opts Bars [width=plot_width, height=plot_height] {+axiswise}
print("Cache simulation: " + cache_simulation_type)
start = timer()
d_s = {'data_name':[], 'thread_id': [], 'property': [], 'value': []}
for data_name in sorted(summary_data.keys()):
    for thread_name in sorted(summary_data[data_name].keys()):
        for property_name in sorted(summary_data[data_name][thread_name].keys()):
            d_s['data_name'].append(data_name)
            d_s['thread_id'].append(thread_name)
            d_s['property'].append(property_name)
            d_s['value'].append(summary_data[data_name][thread_name][property_name])
df_s = pd.DataFrame(data=d_s)
plots = {}
all_threads = []
for data_name in summary_data:
    for thread_name in summary_data[data_name]:
        if thread_name not in all_threads:
            all_threads.append(thread_name)
properties = ["Cache_References", "Reads", "Writes", "L1_Cache_Hits", "L2_Cache_Hits", "L3_Cache_Hits", "L3_Cache_Misses", "Cache_Hits_Modified", "L3_Hits_Shared"]
for thread_name in sorted(all_threads):
    for property_name in properties:
        plots[(thread_name, property_name)] = hv.Bars(df_s.loc[(df_s['thread_id'] == thread_name) & (df_s['property'] == property_name)], kdims=['data_name'], vdims=['value'])
holomap = hv.HoloMap(plots, kdims=['thread_id','property_name'])
end = timer()
print("Time: " + str(end - start) + " s")
holomap

# Plot Cache Sim: Per Cache Level

In [ ]:
%%opts RGB [width=plot_width, height=plot_height] {+axiswise}
print("Cache simulation: " + cache_simulation_type)
start = timer()
cache_levels = []
row_count = {}
data_row = {}
thread_id_to_name = {}
thread_name_to_id = {}
n_threads = 0
data_frames = OrderedDict()
minx = 0
miny = 0
maxx = 0
maxy = 0
elements_per_row = 10
cache_levels = ["Cache_References", "Reads", "Writes", "L3_Hits_Shared", "L1_Cache_Hits", "L2_Cache_Hits", "L3_Cache_Hits", "L3_Cache_Misses", "Cache_Hits_Modified"]
for data_name in data:
    d = OrderedDict()
    for cache_level in cache_levels:
        d[cache_level] = {'thread_id': [], 'address': [], 'data_row': []}
    if data_name not in data_row:
        data_row[data_name] = {}
        row_count[data_name] = {}
    for thread_name in sorted(thread_names[data_name]):
        if thread_name not in thread_name_to_id:
            thread_id_to_name[n_threads] = thread_name
            thread_name_to_id[thread_name] = n_threads
            n_threads += 1
        if thread_name not in data_row[data_name]:
            data_row[data_name][thread_name] = 0
            row_count[data_name][thread_name] = 0
        for cache_level in cache_levels: # Append dummy data to prevent issues with empty data intersections
            d[cache_level]['thread_id'].append(thread_name_to_id[thread_name])
            d[cache_level]['address'].append(0)
            d[cache_level]['data_row'].append(-1)
    for i in range(max_len[data_name]):
        for thread_name in data[data_name][i]:
            address = data[data_name][i][thread_name]
            cache_level = cache_misses[data_name][thread_name][i]
            l = cache_level
            d["Cache_References"]['thread_id'].append(thread_name_to_id[thread_name])
            d["Cache_References"]['address'].append(address)
            d["Cache_References"]['data_row'].append(data_row[data_name][thread_name])
            if l // 100 == 0:
                d["Reads"]['thread_id'].append(thread_name_to_id[thread_name])
                d["Reads"]['address'].append(address)
                d["Reads"]['data_row'].append(data_row[data_name][thread_name])
            else: 
                d["Writes"]['thread_id'].append(thread_name_to_id[thread_name])
                d["Writes"]['address'].append(address)
                d["Writes"]['data_row'].append(data_row[data_name][thread_name])
            l = (l % 100)
            if l // 10 == 1:
                d["L3_Hits_Shared"]['thread_id'].append(thread_name_to_id[thread_name])
                d["L3_Hits_Shared"]['address'].append(address)
                d["L3_Hits_Shared"]['data_row'].append(data_row[data_name][thread_name])
            l = (l % 10)
            if l == 1:
                d["L1_Cache_Hits"]['thread_id'].append(thread_name_to_id[thread_name])
                d["L1_Cache_Hits"]['address'].append(address)
                d["L1_Cache_Hits"]['data_row'].append(data_row[data_name][thread_name])
            elif l == 2:
                d["L2_Cache_Hits"]['thread_id'].append(thread_name_to_id[thread_name])
                d["L2_Cache_Hits"]['address'].append(address)
                d["L2_Cache_Hits"]['data_row'].append(data_row[data_name][thread_name])
            elif l == 3:
                d["L3_Cache_Hits"]['thread_id'].append(thread_name_to_id[thread_name])
                d["L3_Cache_Hits"]['address'].append(address)
                d["L3_Cache_Hits"]['data_row'].append(data_row[data_name][thread_name])
            elif l == 4:
                d["L3_Cache_Misses"]['thread_id'].append(thread_name_to_id[thread_name])
                d["L3_Cache_Misses"]['address'].append(address)
                d["L3_Cache_Misses"]['data_row'].append(data_row[data_name][thread_name])
            elif l == 5:
                d["Cache_Hits_Modified"]['thread_id'].append(thread_name_to_id[thread_name])
                d["Cache_Hits_Modified"]['address'].append(address)
                d["Cache_Hits_Modified"]['data_row'].append(data_row[data_name][thread_name])
            maxx = max(maxx, address)
            minx = min(minx, address)
            row_count[data_name][thread_name] += 1
            if row_count[data_name][thread_name] > elements_per_row:
                data_row[data_name][thread_name] += 1
                row_count[data_name][thread_name] = 0
            maxy = max(maxy, data_row[data_name][thread_name])
    data_frames[data_name] = OrderedDict()
    for cache_level in cache_levels:
        data_frames[data_name][cache_level] = pd.DataFrame(data=d[cache_level])
            
end = timer()
print("Time: " + str(end - start) + " s")
thread_keys = ["Thread " + str(i) for i in range(n_threads)]
color_key = OrderedDict((k, c) for k, c in zip(thread_keys, Sets1to3[0:len(thread_keys)]))
colors = hv.NdOverlay(OrderedDict((c, hv.Points([0,0], label=c).opts(style=dict(color=color_key[c]))) for c in thread_keys))
scatter_dict = OrderedDict()
opts = dict(x_range=(minx, maxx), y_range=(miny, maxy))
for data_name in data:
    for cache_level in cache_levels:
        df = data_frames[data_name][cache_level]
        plot = OrderedDict((k, hv.Scatter(df.loc[(df['thread_id'] == k)], kdims=['address', 'data_row'])) for k in range(n_threads))
        plot_data = hv.NdOverlay(plot, kdims='thread_keys')
        scatter_dict[(data_name, cache_level)] = plot_data
hmap = dynspread(datashade(hv.HoloMap(scatter_dict, kdims=['data_name', 'cache_level']), aggregator=ds.count_cat('thread_keys'), **opts), threshold=0.75, how='over') * colors
end = timer()
print("Time: " + str(end - start) + " s")
hmap

# Plot Cache Sim: All Cache Levels

In [ ]:
%%opts RGB [width=plot_width, height=plot_height] {+axiswise}
print("Cache simulation: " + cache_simulation_type)
start = timer()
row_count = {}
thread_id_to_name = {}
thread_name_to_id = {}
data_row = {}
n_threads = 0
data_frames = OrderedDict()
minx = 0
miny = 0
maxx = 0
maxy = 0
elements_per_row = 10
for data_name in data:
    d = OrderedDict()
    for thread_name in thread_names[data_name] + ["All"]:
        d[thread_name] = {'address': [], 'data_row': [], 'cache_level': []}
    if data_name not in data_row:
        data_row[data_name] = {}
        row_count[data_name] = {}
    for thread_name in thread_names[data_name] + ["All"]:
        n_threads += 1
        if thread_name not in data_row[data_name]:
            data_row[data_name][thread_name] = 0
            row_count[data_name][thread_name] = 0
        for hit in [1, 2, 3, 4, 5, 6]: # Append dummy data to prevent issues with empty data intersections
            d[thread_name]['cache_level'].append(hit)
            d[thread_name]['address'].append(0)
            d[thread_name]['data_row'].append(-1)
    for i in range(max_len[data_name]):
        for thread_name in data[data_name][i]:
            address = data[data_name][i][thread_name]
            maxx = max(maxx, address)
            cache_level = cache_misses[data_name][thread_name][i]
            l = cache_level
            l = (l % 100)
            m = (l % 10)
            if m == 5:
                hit = 6
            elif m == 1:
                hit = 1
            elif m == 2:
                hit = 2
            elif l // 10 == 1:
                hit = 3
            elif m == 3:
                hit = 4
            elif m == 4:
                hit = 5
            else:
                print("No hit: ", str(cache_level))
                continue
            d[thread_name]['address'].append(address)
            d[thread_name]['data_row'].append(data_row[data_name][thread_name])
            d[thread_name]['cache_level'].append(hit)
            d["All"]['address'].append(address)
            d["All"]['data_row'].append(data_row[data_name][thread_name])  
            d["All"]['cache_level'].append(hit)
            minx = min(minx, address)
            row_count[data_name][thread_name] += 1
            if row_count[data_name][thread_name] > elements_per_row:
                data_row[data_name][thread_name] += 1
                row_count[data_name][thread_name] = 0
            maxy = max(maxy, data_row[data_name][thread_name])
    data_frames[data_name] = OrderedDict()
    for thread_name in thread_names[data_name] + ["All"]:
        data_frames[data_name][thread_name] = pd.DataFrame(data=d[thread_name])
end = timer()
print("Time: " + str(end - start) + " s")
cache_levels = [1, 2, 3, 4, 5, 6]
cache_level_keys = ["L1 Hit", "L2 Hit", "L3 Hit Shared", "L3 Hit", "L3 Miss", "Modified"]
color_key = OrderedDict((k, c) for k, c in zip(cache_level_keys, Sets1to3[0:len(cache_level_keys)]))
colors = hv.NdOverlay(OrderedDict((c, hv.Points([0,0], label=c).opts(style=dict(color=color_key[c]))) for c in cache_level_keys))
scatter_dict = OrderedDict()
opts = dict(x_range=(minx, maxx), y_range=(miny, maxy))
threads = [str(th) for th in sorted(thread_names[data_name], key=lambda x: int(x))] + ["All"]
for data_name in data:
    for thread_name in threads:
        df = data_frames[data_name][thread_name]
        plot = OrderedDict((cache_level, hv.Scatter(df.loc[(df['cache_level'] == cache_level)], kdims=['address', 'data_row'])) for cache_level in cache_levels)
        plot_data = hv.NdOverlay(plot, kdims='cache_level_keys')
        scatter_dict[(data_name, thread_name)] = plot_data
hmap = dynspread(datashade(hv.HoloMap(scatter_dict, kdims=['data_name', 'thread_name']), aggregator=ds.count_cat('cache_level_keys'), **opts), threshold=0.75, how='over') * colors
end = timer()
print("Time: " + str(end - start) + " s")
hmap

# Last Access by Address

In [ ]:
%%opts RGB [width=plot_width, height=plot_height] {+axiswise}
print("Cache simulation: " + cache_simulation_type)
start = timer()
last_address = {}
thread_id_to_name = {}
thread_name_to_id = {}
last_access = {}
n_threads = 0
data_frames = OrderedDict()
minx = 0
miny = 0
maxx = 0
maxy = 0
for data_name in data:
    d = OrderedDict()
    for thread_name in thread_names[data_name] + ["All"]:
        d[thread_name] = {'address': [], 'last_access': [], 'cache_level': []}
    if data_name not in last_access:
        last_access[data_name] = {}
    for thread_name in thread_names[data_name] + ["All"]:
        n_threads += 1
        if thread_name not in last_access[data_name]:
            last_access[data_name][thread_name] = {}
    for i in range(max_len[data_name]):
        for thread_name in data[data_name][i]:
            address = data[data_name][i][thread_name]
            cache_level = cache_misses[data_name][thread_name][i]
            if address in last_access[data_name][thread_name]:
                dl = i - last_access[data_name][thread_name][address]
            else:
                dl = 0
            last_access[data_name][thread_name][address] = i
            maxx = max(maxx, address)
            minx = min(minx, address)
            l = cache_level
            l = (l % 100)
            m = (l % 10)
            if m == 5:
                hit = 6
            elif m == 1:
                hit = 1
            elif m == 2:
                hit = 2
            elif l // 10 == 1:
                hit = 3
            elif m == 3:
                hit = 4
            elif m == 4:
                hit = 5
            else:
                print("No hit: ", str(cache_level))
                continue
            d[thread_name]['address'].append(address)
            d[thread_name]['last_access'].append(dl)
            d[thread_name]['cache_level'].append(hit)
            d["All"]['address'].append(address)
            d["All"]['last_access'].append(dl)  
            d["All"]['cache_level'].append(hit)
            maxy = max(maxy, dl)
    data_frames[data_name] = OrderedDict()
    for thread_name in thread_names[data_name] + ["All"]:
        data_frames[data_name][thread_name] = pd.DataFrame(data=d[thread_name])
end = timer()
print("Time: " + str(end - start) + " s")
cache_levels = [1, 2, 3, 4, 5, 6]
cache_level_keys = ["L1 Hit", "L2 Hit", "L3 Hit Shared", "L3 Hit", "L3 Miss", "Modified"]
color_key = OrderedDict((k, c) for k, c in zip(cache_level_keys, Sets1to3[0:len(cache_level_keys)]))
colors = hv.NdOverlay(OrderedDict((c, hv.Points([0,0], label=c).opts(style=dict(color=color_key[c]))) for c in cache_level_keys))
scatter_dict = OrderedDict()
opts = dict(x_range=(minx, maxx), y_range=(miny, maxy))
threads = [str(th) for th in sorted(thread_names[data_name], key=lambda x: int(x))] + ["All"]
for data_name in data:
    for thread_name in threads:
        df = data_frames[data_name][thread_name]
        plot = OrderedDict((cache_level, hv.Scatter(df.loc[(df['cache_level'] == cache_level)], kdims=['address', 'last_access'])) for cache_level in cache_levels)
        plot_data = hv.NdOverlay(plot, kdims='cache_level_keys')
        scatter_dict[(data_name, thread_name)] = plot_data
hmap = dynspread(datashade(hv.HoloMap(scatter_dict, kdims=['data_name', 'thread_name']), aggregator=ds.count_cat('cache_level_keys'), **opts), threshold=0.75, how='over') * colors
end = timer()
print("Time: " + str(end - start) + " s")
hmap

# Data Access Count

In [ ]:
%%opts RGB [width=plot_width, height=plot_height] {+axiswise}
print("Cache simulation: " + cache_simulation_type)
start = timer()
last_address = {}
data_row = {}
thread_id_to_name = {}
thread_name_to_id = {}
data_count = {}
n_threads = 0
data_frames = OrderedDict()
minx = 0
miny = 0
maxx = 0
maxy = 0
for data_name in data:
    if data_name not in data_count:
        data_count[data_name] = {}
    for thread_name in thread_names[data_name] + ["All"]:
        n_threads += 1
        if thread_name not in data_count[data_name]:
            data_count[data_name][thread_name] = {}
    for i in range(max_len[data_name]):
        for thread_name in data[data_name][i]:
            address = data[data_name][i][thread_name]
            cache_level = cache_misses[data_name][thread_name][i]
            maxx = max(maxx, address)
            minx = min(minx, address)
            l = cache_level
            l = (l % 100)
            m = (l % 10)
            if m == 5:
                hit = 6
            elif m == 1:
                hit = 1
            elif m == 2:
                hit = 2
            elif l // 10 == 1:
                hit = 3
            elif m == 3:
                hit = 4
            elif m == 4:
                hit = 5
            else:
                print("No hit: ", str(cache_level))
                continue
            data_key = str(hit) + ":" + str(address) 
            if data_key in data_count[data_name][thread_name]:
                data_count[data_name][thread_name][data_key] += 1
            else:
                data_count[data_name][thread_name][data_key] = 0
        
for data_name in data:
    d = OrderedDict()
    for thread_name in thread_names[data_name] + ["All"]:
        d[thread_name] = {'address': [], 'count': [], 'cache_level': []}
    if data_name not in data_count:
        data_count[data_name] = {}
    for thread_name in thread_names[data_name] + ["All"]:
        n_threads += 1
        if thread_name not in data_count[data_name]:
            data_count[data_name][thread_name] = {}
    for thread_name in data_count[data_name]:
        for data_key in data_count[data_name][thread_name]:
            hit, par, address = data_key.partition(":")
            count = data_count[data_name][thread_name][data_key]
            d[thread_name]['address'].append(int(address))
            d[thread_name]['count'].append(count)
            d[thread_name]['cache_level'].append(int(hit))
            d["All"]['address'].append(int(address))
            d["All"]['count'].append(count)  
            d["All"]['cache_level'].append(int(hit))
            maxy = max(maxy, count)
    data_frames[data_name] = OrderedDict()
    for thread_name in thread_names[data_name] + ["All"]:
        data_frames[data_name][thread_name] = pd.DataFrame(data=d[thread_name])
        
        
end = timer()
print("Time: " + str(end - start) + " s")
cache_levels = [1, 2, 3, 4, 5, 6]
cache_level_keys = ["L1 Hit", "L2 Hit", "L3 Hit Shared", "L3 Hit", "L3 Miss", "Modified"]
color_key = OrderedDict((k, c) for k, c in zip(cache_level_keys, Sets1to3[0:len(cache_level_keys)]))
colors = hv.NdOverlay(OrderedDict((c, hv.Points([0,0], label=c).opts(style=dict(color=color_key[c]))) for c in cache_level_keys))
scatter_dict = OrderedDict()
opts = dict(x_range=(minx, maxx), y_range=(miny, maxy))
threads = [str(th) for th in sorted(thread_names[data_name], key=lambda x: int(x))] + ["All"]
for data_name in data:
    for thread_name in threads:
        df = data_frames[data_name][thread_name]
        plot = OrderedDict((cache_level, hv.Scatter(df.loc[(df['cache_level'] == cache_level)], kdims=['address', 'count'])) for cache_level in cache_levels)
        plot_data = hv.NdOverlay(plot, kdims='cache_level_keys')
        scatter_dict[(data_name, thread_name)] = plot_data
hmap = dynspread(datashade(hv.HoloMap(scatter_dict, kdims=['data_name', 'thread_name']), aggregator=ds.count_cat('cache_level_keys'), **opts), threshold=0.75, how='over') * colors
end = timer()
print("Time: " + str(end - start) + " s")
hmap